In [1]:
import os
os.chdir("../")
%pwd

'd:\\DE\\ML OPs\\DSusingMLOps'

In [2]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd

In [3]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    model_name: str
    train_date_path: Path
    test_data_path: Path
    alpha: float
    l1_ratio: float
    target_column: str

In [4]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager():
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                                params_filepath= PARAMS_FILE_PATH,
                                schema_filepath=SCHEMA_FILE_PATH ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_tainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            model_name=config.model_name,
            train_date_path=Path(config.train_date_path),
            test_data_path=Path(config.test_data_path),
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=self.schema.name
        )
        return model_trainer_config
        

In [ ]:
import pandas as pd
import os
from src.datascience.utils.common import save_json,load_json